In [ ]:
import taichi as ti

ti.init(
    arch=ti.cpu,
    default_fp=ti.f64,
    offline_cache=False,
    debug=True,
    cpu_max_num_threads=1,
)

In [ ]:
from functools import partial

import numpy as np
import bilby
import lal
import lalsimulation as lalsim
from matplotlib import pyplot as plt

%matplotlib inline

from common_funcs import is_equal, mismatch_precession_spins, plot_waveform_comparison

In [ ]:
reference_frequency = 20.0
minimum_frequency = 20.0
maximum_frequency = 2048.0
sampling_rate = 4096
duration = 32.0

long_asc_nodes = 0.0
eccentricity = 0.0
mean_per_ano = 0.0

num_samples = int(duration * sampling_rate)
full_freqs = np.fft.rfftfreq(num_samples, 1 / sampling_rate)
freqs_mask = (full_freqs <= maximum_frequency) * (full_freqs >= minimum_frequency)
freqs = full_freqs[freqs_mask]
delta_f = freqs[1] - freqs[0]
freqs_ti = ti.field(ti.f64, shape=freqs.shape)
freqs_ti.from_numpy(freqs)

# psd_array = np.ones(shape=freqs.shape)
PSD_Aplus = bilby.gw.detector.PowerSpectralDensity(asd_file="Aplus_asd.txt")
psd_array = PSD_Aplus.get_power_spectral_density_array(freqs)

## 1. Comparing values of ansatz coefficients (TL; DR)

In [ ]:
from tiwave.waveforms import IMRPhenomXP

In [ ]:
params_prec = dict(
    mass_1=36.0,
    mass_2=29.0,
    luminosity_distance=800.0,
    reference_phase=1.2,
    # chi_1=-0.4,
    # chi_2=0.02,
    # inclination=0.4,
    a_1=0.4,
    a_2=0.3,
    tilt_1=0.5,
    tilt_2=1.0,
    phi_12=1.7,
    phi_jl=0.3,
    theta_jn=0.4,
)
(
    params_prec["inclination"],
    params_prec["chi1_x"],
    params_prec["chi1_y"],
    params_prec["chi1_z"],
    params_prec["chi2_x"],
    params_prec["chi2_y"],
    params_prec["chi2_z"],
) = bilby.gw.conversion.bilby_to_lalsimulation_spins(
    params_prec["theta_jn"],
    params_prec["phi_jl"],
    params_prec["tilt_1"],
    params_prec["tilt_2"],
    params_prec["phi_12"],
    params_prec["a_1"],
    params_prec["a_2"],
    params_prec["mass_1"],
    params_prec["mass_2"],
    reference_frequency,
    params_prec["reference_phase"],
)

In [ ]:
xp_tiw = IMRPhenomXP(
    freqs_ti,
    reference_frequency,
    return_form="polarizations",
    precession_model="NNLO",
)
xp_tiw.update_waveform(params_prec)

In [ ]:
# fmt: off
lalsim_XP_coeffs = dict(
    afinal = 7.6807644664650421e-01,
    fring  = 9.5550665779054286e-02,
    fdamp  = 1.3091098608325224e-02,

    alpha_offset   = -1.6061244860838741e01,
    epsilon_offset = -9.9749806142079969e00,
    zeta           = -2.9950142336587136e00,

    alpha1  = -1.6710069444444442e-01,
    alpha2  = -1.4985318553846896e-01,
    alpha3  = -1.9858053358738070e00,
    alpha4L = -1.6393171066938781e00,
    alpha5  = 1.1636843719159891e00,

    epsilon1  = -1.6710069444444442e-01,
    epsilon2  = -1.4985318553846896e-01,
    epsilon3  = -1.9707855966983070e00,
    epsilon4L = -1.6363238938371516e00,
    epsilon5  = 1.1487535486069940e00,

    L0 = 1.0000000000000000e00,
    L1 = 0.0000000000000000e00,
    L2 = 1.5411834319526627e00,
    L3 = -7.8345794231715316e-01,
    L4 = 2.7906802072756558e00,
    L5 = -1.1168285799113600e00,
    L6 = 8.6134589361196323e-01,

    Y2m2 = 1.2202739888943499e-03,
    Y2m1 = 1.1378257263392413e-02,
    Y20  = 6.4969537892986776e-02,
    Y21  = 2.4731619007002753e-01,
    Y22  = 5.7651545096206758e-01,
)
# fmt: on

In [ ]:
# fmt: off
is_equal(lalsim_XP_coeffs['afinal'], xp_tiw.source_parameters[None].final_spin, "final_spin")
is_equal(lalsim_XP_coeffs['fring'],  xp_tiw.source_parameters[None].f_ring, "f_ring")
is_equal(lalsim_XP_coeffs['fdamp'],  xp_tiw.source_parameters[None].f_damp, "f_damp")

is_equal(-lalsim_XP_coeffs['alpha_offset'],  xp_tiw.precession_coefficients[None].alpha_0 - xp_tiw.precession_coefficients[None].alpha_log * np.log(np.pi), "alpha_0")
is_equal(lalsim_XP_coeffs['epsilon_offset'], xp_tiw.precession_coefficients[None].gamma_0 - xp_tiw.precession_coefficients[None].gamma_log * np.log(np.pi), "gamma_0")
is_equal(np.cos(2.0*lalsim_XP_coeffs['zeta']),     xp_tiw.precession_coefficients[None].cos_2zeta, "cos_2zeta")
is_equal(np.sin(2.0*lalsim_XP_coeffs['zeta']),     xp_tiw.precession_coefficients[None].sin_2zeta, "sin_2zeta")

is_equal(lalsim_XP_coeffs['alpha1']/np.pi,         xp_tiw.precession_coefficients[None].alpha_m3, "alpha_m3")
is_equal(lalsim_XP_coeffs['alpha2']/np.pi**(2/3),  xp_tiw.precession_coefficients[None].alpha_m2, "alpha_m2")
is_equal(lalsim_XP_coeffs['alpha3']/np.pi**(1/3),  xp_tiw.precession_coefficients[None].alpha_m1, "alpha_m1")
is_equal(lalsim_XP_coeffs['alpha4L'],              xp_tiw.precession_coefficients[None].alpha_log, "alpha_log")
is_equal(lalsim_XP_coeffs['alpha5']*np.pi**(1/3),  xp_tiw.precession_coefficients[None].alpha_1, "alpha_1")

is_equal(-lalsim_XP_coeffs['epsilon1']/np.pi,         xp_tiw.precession_coefficients[None].gamma_m3, "gamma_m3")
is_equal(-lalsim_XP_coeffs['epsilon2']/np.pi**(2/3),  xp_tiw.precession_coefficients[None].gamma_m2, "gamma_m2")
is_equal(-lalsim_XP_coeffs['epsilon3']/np.pi**(1/3),  xp_tiw.precession_coefficients[None].gamma_m1, "gamma_m1")
is_equal(-lalsim_XP_coeffs['epsilon4L'],              xp_tiw.precession_coefficients[None].gamma_log, "gamma_log")
is_equal(-lalsim_XP_coeffs['epsilon5']*np.pi**(1/3),  xp_tiw.precession_coefficients[None].gamma_1, "gamma_1")

is_equal(lalsim_XP_coeffs['L0']/np.pi**(1/3), xp_tiw.precession_coefficients[None].L_0, "L_0")
is_equal(lalsim_XP_coeffs['L1'],              xp_tiw.precession_coefficients[None].L_1, "L_1")
is_equal(lalsim_XP_coeffs['L2']*np.pi**(1/3), xp_tiw.precession_coefficients[None].L_2, "L_2")
is_equal(lalsim_XP_coeffs['L3']*np.pi**(2/3), xp_tiw.precession_coefficients[None].L_3, "L_3")
is_equal(lalsim_XP_coeffs['L4']*np.pi,        xp_tiw.precession_coefficients[None].L_4, "L_4")
is_equal(lalsim_XP_coeffs['L5']*np.pi**(4/3), xp_tiw.precession_coefficients[None].L_5, "L_5")
is_equal(lalsim_XP_coeffs['L6']*np.pi**(5/3), xp_tiw.precession_coefficients[None].L_6, "L_6")

is_equal(lalsim_XP_coeffs['Y2m2'], xp_tiw.precession_coefficients[None].spin_minus2_Y2m[0], "Y2m2")
is_equal(lalsim_XP_coeffs['Y2m1'], xp_tiw.precession_coefficients[None].spin_minus2_Y2m[1], "Y2m1")
is_equal(lalsim_XP_coeffs['Y20'],  xp_tiw.precession_coefficients[None].spin_minus2_Y2m[2], "Y20")
is_equal(lalsim_XP_coeffs['Y21'],  xp_tiw.precession_coefficients[None].spin_minus2_Y2m[3], "Y21")
is_equal(lalsim_XP_coeffs['Y22'],  xp_tiw.precession_coefficients[None].spin_minus2_Y2m[4], "Y22")
# fmt: on

## 2. Comparing the waveform visually

In [ ]:
appro_xp = lalsim.GetApproximantFromString("IMRPhenomXP")
extra_params = lal.CreateDict()
lalsim.SimInspiralWaveformParamsInsertPhenomXPrecVersion(extra_params, 102)

hp_lal, hc_lal = lalsim.SimInspiralChooseFDWaveform(
    float(params_prec["mass_1"] * lal.MSUN_SI),
    float(params_prec["mass_2"] * lal.MSUN_SI),
    float(params_prec["chi1_x"]),
    float(params_prec["chi1_y"]),
    float(params_prec["chi1_z"]),
    float(params_prec["chi2_x"]),
    float(params_prec["chi2_y"]),
    float(params_prec["chi2_z"]),
    float(params_prec["luminosity_distance"] * 1e6 * lal.PC_SI),
    float(params_prec["inclination"]),
    float(params_prec["reference_phase"]),
    float(long_asc_nodes),
    float(eccentricity),
    float(mean_per_ano),
    float(delta_f),
    float(freqs[0]),
    float(freqs[-1]),
    float(reference_frequency),
    extra_params,
    appro_xp,
)

fig = plot_waveform_comparison(
    freqs,
    xp_tiw.waveform_container_numpy,
    {"plus": hp_lal.data.data[freqs_mask], "cross": hc_lal.data.data[freqs_mask]},
)
fig.tight_layout()

## 3. Mismatch in the whole parameter space

In [ ]:
xp_tiw = IMRPhenomXP(
    freqs_ti,
    reference_frequency,
    return_form="polarizations",
    precession_model="NNLO",
)

appro_xp = lalsim.GetApproximantFromString("IMRPhenomXP")
extra_params = lal.CreateDict()
lalsim.SimInspiralWaveformParamsInsertPhenomXPrecVersion(extra_params, 102)


def xp_tiw_wrapper(
    mass_1,
    mass_2,
    chi1_x,
    chi1_y,
    chi1_z,
    chi2_x,
    chi2_y,
    chi2_z,
    luminosity_distance,
    inclination,
    reference_phase,
):
    params_in = {
        "mass_1": mass_1,
        "mass_2": mass_2,
        "chi1_x": chi1_x,
        "chi1_y": chi1_y,
        "chi1_z": chi1_z,
        "chi2_x": chi2_x,
        "chi2_y": chi2_y,
        "chi2_z": chi2_z,
        "luminosity_distance": luminosity_distance,
        "inclination": inclination,
        "reference_phase": reference_phase,
    }
    xp_tiw.update_waveform(params_in)
    return xp_tiw.waveform_container_numpy


def xp_lal_wrapper(
    mass_1,
    mass_2,
    chi1_x,
    chi1_y,
    chi1_z,
    chi2_x,
    chi2_y,
    chi2_z,
    luminosity_distance,
    inclination,
    reference_phase,
):
    hp, hc = lalsim.SimInspiralChooseFDWaveform(
        float(mass_1 * lal.MSUN_SI),
        float(mass_2 * lal.MSUN_SI),
        float(chi1_x),
        float(chi1_y),
        float(chi1_z),
        float(chi2_x),
        float(chi2_y),
        float(chi2_z),
        float(luminosity_distance * 1e6 * lal.PC_SI),
        float(inclination),
        float(reference_phase),
        float(long_asc_nodes),
        float(eccentricity),
        float(mean_per_ano),
        float(delta_f),
        float(freqs[0]),
        float(freqs[-1]),
        float(reference_frequency),
        extra_params,
        appro_xp,
    )
    return {"plus": hp.data.data[freqs_mask], "cross": hc.data.data[freqs_mask]}


mismatch_xp = partial(
    mismatch_precession_spins,
    tiw_wf_func=xp_tiw_wrapper,
    lal_wf_func=xp_lal_wrapper,
    psd_array=psd_array,
    delta_f=delta_f,
)

In [ ]:
print(
    "Mismatch of the test case: ",
    mismatch_xp(
        params_prec["mass_1"],
        params_prec["mass_2"],
        params_prec["chi1_x"],
        params_prec["chi1_y"],
        params_prec["chi1_z"],
        params_prec["chi2_x"],
        params_prec["chi2_y"],
        params_prec["chi2_z"],
        params_prec["luminosity_distance"],
        params_prec["inclination"],
        params_prec["reference_phase"],
    ),
)

In [ ]:
num = 10000
rng = np.random.default_rng(seed=20250305)

# samples of precession spins
samples_prec = {}
samples_prec["total_mass"] = rng.uniform(20, 250, num)
# X family models are calibrated to mass ratio from 1 to 1000, using log-uniform to cover more case with extreme mass ratio
samples_prec["mass_ratio"] = 10 ** rng.uniform(-3, 0, num)
samples_prec = bilby.gw.conversion.generate_component_masses(samples_prec)
samples_prec["reference_phase"] = rng.uniform(0, 2 * np.pi, num)
samples_prec["luminosity_distance"] = params_prec["luminosity_distance"] * np.ones(num)
# samples_x['luminosity_distance'] = rng.uniform(20, 1000, num) * np.ones(num)

# samples_align["chi_1"] = rng.uniform(-0.99, 0.99, num)
# samples_align["chi_2"] = rng.uniform(-0.99, 0.99, num)
# samples_align["cos_iota"] = rng.uniform(-1, 1, num)
# samples_align["inclination"] = np.arccos(samples_align["cos_iota"])

samples_prec["theta_jn"] = np.arccos(rng.uniform(-1, 1, num))
samples_prec["a_1"] = rng.uniform(0.0, 0.99, num)
samples_prec["a_2"] = rng.uniform(0.0, 0.99, num)
samples_prec["tilt_1"] = np.arccos(rng.uniform(-1, 1, num))
samples_prec["tilt_2"] = np.arccos(rng.uniform(-1, 1, num))
samples_prec["phi_12"] = rng.uniform(0.0, 2.0 * np.pi, num)
samples_prec["phi_jl"] = rng.uniform(0.0, 2.0 * np.pi, num)

(
    samples_prec["inclination"],
    samples_prec["chi1_x"],
    samples_prec["chi1_y"],
    samples_prec["chi1_z"],
    samples_prec["chi2_x"],
    samples_prec["chi2_y"],
    samples_prec["chi2_z"],
) = np.vectorize(bilby.gw.conversion.bilby_to_lalsimulation_spins)(
    samples_prec["theta_jn"],
    samples_prec["phi_jl"],
    samples_prec["tilt_1"],
    samples_prec["tilt_2"],
    samples_prec["phi_12"],
    samples_prec["a_1"],
    samples_prec["a_2"],
    samples_prec["mass_1"],
    samples_prec["mass_2"],
    reference_frequency,
    samples_prec["reference_phase"],
)

# print(samples_prec)
# print(samples_prec.keys())
# check_1 = (
#     np.sqrt(
#         samples_prec["chi1_x"] ** 2
#         + samples_prec["chi1_y"] ** 2
#         + samples_prec["chi1_z"] ** 2
#     )
#     - samples_prec["a_1"]
# )
# check_2 = (
#     np.sqrt(
#         samples_prec["chi2_x"] ** 2
#         + samples_prec["chi2_y"] ** 2
#         + samples_prec["chi2_z"] ** 2
#     )
#     - samples_prec["a_2"]
# )
# print(np.max(np.abs(check_1)))
# print(np.max(np.abs(check_2)))

In [ ]:
mism = list(
    map(
        mismatch_xp,
        samples_prec["mass_1"],
        samples_prec["mass_2"],
        samples_prec["chi1_x"],
        samples_prec["chi1_y"],
        samples_prec["chi1_z"],
        samples_prec["chi2_x"],
        samples_prec["chi2_y"],
        samples_prec["chi2_z"],
        samples_prec["luminosity_distance"],
        samples_prec["inclination"],
        samples_prec["reference_phase"],
    )
)
mism = np.array(mism)

mism_p = mism[:, 0]
mism_c = mism[:, 1]

In [ ]:
print("Max mismatch hp: ", mism_p.max())
print("Min mismatch hp: ", mism_p.min())
print("Max mismatch hc: ", mism_c.max())
print("Min mismatch hc: ", mism_c.min())

# note minus values are induced by numerical errors in the computation of the inner product
fig, ax = plt.subplots()
ax.hist(
    mism_p,
    bins=50,
    histtype="step",
    label="plus",
)
ax.hist(
    mism_c,
    bins=50,
    histtype="step",
    label="cross",
)
ax.set_xlabel(r"$1-\langle \hat{h}^{\rm tiw} , \hat{h}^{\rm lal}\rangle$")
ax.set_ylabel("Num.")
ax.legend()

In [ ]:
nonzero_mm_p = np.abs(mism_p) > 0
nonzero_mm_c = np.abs(mism_c) > 0

fig, ax = plt.subplots()
ax.hist(
    np.log10(np.abs(mism_p)[nonzero_mm_p]),
    bins=50,
    histtype="step",
    label="plus",
)
ax.hist(
    np.log10(np.abs(mism_c)[nonzero_mm_c]),
    bins=50,
    histtype="step",
    label="cross",
)
ax.set_xlabel(r"$\log_{10}|1-\langle \hat{h}^{\rm tiw} , \hat{h}^{\rm lal}\rangle|$")
ax.set_ylabel("Num.")
ax.legend()

In [ ]:
samples_prec["log10_mass_ratio"] = np.log10(samples_prec["mass_ratio"])

params_names = [
    "total_mass",
    "log10_mass_ratio",
    "a_1",
    "a_2",
    "tilt_1",
    "tilt_2",
    "phi_12",
    "phi_jl",
    "theta_jn",
    "reference_phase",
]
num_params = len(params_names)

# plus
fig, axes = plt.subplots(
    num_params, num_params, figsize=(3 * (num_params - 1), 3 * (num_params - 1))
)
plt.subplots_adjust(hspace=0.05, wspace=0.05)

for i in range(num_params):
    for j in range(num_params):
        ax = axes[i, j]
        ax.margins(0)

        if i <= j:
            ax.axis("off")
        else:
            scatter_handle = ax.scatter(
                samples_prec[params_names[j]][nonzero_mm_p],
                samples_prec[params_names[i]][nonzero_mm_p],
                s=5,
                c=np.log10(np.abs(mism_p)[nonzero_mm_p]),
            )

        if i == num_params - 1:
            ax.set_xlabel(params_names[j])
        else:
            ax.set_xticks([])

        if j == 0:
            ax.set_ylabel(params_names[i])
        else:
            ax.set_yticks([])

fig.colorbar(
    scatter_handle,
    ax=axes,
    orientation="vertical",
    fraction=0.02,
    pad=-0.2,
    label=r"$\log_{10}|1-\langle \hat{h}^{\rm tiw},\ \hat{h}^{\rm lal}\rangle|$",
)

fig.suptitle(r"Mismatch of $h_+$", y=0.8, fontsize=64)

# cross
fig, axes = plt.subplots(
    num_params, num_params, figsize=(3 * (num_params - 1), 3 * (num_params - 1))
)
plt.subplots_adjust(hspace=0.05, wspace=0.05)

for i in range(num_params):
    for j in range(num_params):
        ax = axes[i, j]
        ax.margins(0)

        if i <= j:
            ax.axis("off")
        else:
            scatter_handle = ax.scatter(
                samples_prec[params_names[j]][nonzero_mm_c],
                samples_prec[params_names[i]][nonzero_mm_c],
                s=5,
                c=np.log10(np.abs(mism_c)[nonzero_mm_c]),
            )

        if i == num_params - 1:
            ax.set_xlabel(params_names[j])
        else:
            ax.set_xticks([])

        if j == 0:
            ax.set_ylabel(params_names[i])
        else:
            ax.set_yticks([])

fig.colorbar(
    scatter_handle,
    ax=axes,
    orientation="vertical",
    fraction=0.02,
    pad=-0.2,
    label=r"$\log_{10}|1-\langle \hat{h}^{\rm tiw},\ \hat{h}^{\rm lal}\rangle|$",
)

fig.suptitle(r"Mismatch of $h_\times$", y=0.8, fontsize=64)

In [ ]:
max_idx_p = np.argmax(np.abs(mism_p))
max_idx_c = np.argmax(np.abs(mism_c))
check_params_p = {
    "mass_1": samples_prec["mass_1"][max_idx_p],
    "mass_2": samples_prec["mass_2"][max_idx_p],
    "chi1_x": samples_prec["chi1_x"][max_idx_p],
    "chi1_y": samples_prec["chi1_y"][max_idx_p],
    "chi1_z": samples_prec["chi1_z"][max_idx_p],
    "chi2_x": samples_prec["chi2_x"][max_idx_p],
    "chi2_y": samples_prec["chi2_y"][max_idx_p],
    "chi2_z": samples_prec["chi2_z"][max_idx_p],
    "luminosity_distance": samples_prec["luminosity_distance"][max_idx_p],
    "inclination": samples_prec["inclination"][max_idx_p],
    "reference_phase": samples_prec["reference_phase"][max_idx_p],
}
check_params_c = {
    "mass_1": samples_prec["mass_1"][max_idx_c],
    "mass_2": samples_prec["mass_2"][max_idx_c],
    "chi1_x": samples_prec["chi1_x"][max_idx_c],
    "chi1_y": samples_prec["chi1_y"][max_idx_c],
    "chi1_z": samples_prec["chi1_z"][max_idx_c],
    "chi2_x": samples_prec["chi2_x"][max_idx_c],
    "chi2_y": samples_prec["chi2_y"][max_idx_c],
    "chi2_z": samples_prec["chi2_z"][max_idx_c],
    "luminosity_distance": samples_prec["luminosity_distance"][max_idx_c],
    "inclination": samples_prec["inclination"][max_idx_c],
    "reference_phase": samples_prec["reference_phase"][max_idx_c],
}
print(max_idx_p)
print(max_idx_c)
print("params for max mismatch of hp: ", check_params_p)
print("params for max mismatch of hc: ", check_params_c)

In [ ]:
xp_tiw = IMRPhenomXP(
    freqs_ti,
    reference_frequency,
    return_form="polarizations",
    precession_model="NNLO",
)

xp_tiw.update_waveform(check_params_p)

# fmt: off
lalsim_coeffs_check_p = dict(
    chiL = 2.0951732608515003e-01,
    chip = 9.3348254149888910e-01,

    afinal = 9.5548917034323633e-01,
    fring  = 1.2058706855740561e-01,
    fdamp  = 8.1517584277483353e-03,

    alpha_offset   = -6.4362179918462934e+09,
    epsilon_offset = -7.6891273003614333e+02,
    zeta           = -2.2579838091766109e+00,

    alpha1  = -1.0424582361437844e-01,
    alpha2  = -3.2334850528667836e+01,
    alpha3  = -1.3272969760199825e+05,
    alpha4L = -9.1487503393128067e+06,
    alpha5  = -2.2489959325358120e+10,

    epsilon1  = -1.0424582361437844e-01,
    epsilon2  = -3.2334850528667836e+01,
    epsilon3  = -1.1611528007077883e+00,
    epsilon4L = 4.6520884621943772e+01,
    epsilon5  = -6.8221175335633325e+02,

    L0 = 1.0000000000000000e+00,
    L1 = 0.0000000000000000e+00,
    L2 = 1.5001685264770472e+00,
    L3 = -6.9746548289552068e-01,
    L4 = 3.3725985403038368e+00,
    L5 = -1.4633507595821307e+00,
    L6 = 8.4061759188215923e+00,

    Y2m2 = 2.0059148615674462e-02,
    Y2m1 = 8.6116027950500995e-02,
    Y20  = 2.2639723471221368e-01,
    Y21  = 3.9679572707648697e-01,
    Y22  = 4.2587141381535126e-01,
)

is_equal(lalsim_coeffs_check_p['chiL'], xp_tiw.source_parameters[None].chi_l, "chi_l")
is_equal(lalsim_coeffs_check_p['chip'], xp_tiw.source_parameters[None].chi_p, "chi_p")

is_equal(lalsim_coeffs_check_p['afinal'], xp_tiw.source_parameters[None].final_spin, "final_spin")
is_equal(lalsim_coeffs_check_p['fring'],  xp_tiw.source_parameters[None].f_ring, "f_ring")
is_equal(lalsim_coeffs_check_p['fdamp'],  xp_tiw.source_parameters[None].f_damp, "f_damp")

is_equal(-lalsim_coeffs_check_p['alpha_offset'],  xp_tiw.precession_coefficients[None].alpha_0 - xp_tiw.precession_coefficients[None].alpha_log * np.log(np.pi), "alpha_0")
is_equal(lalsim_coeffs_check_p['epsilon_offset'], xp_tiw.precession_coefficients[None].gamma_0 - xp_tiw.precession_coefficients[None].gamma_log * np.log(np.pi), "gamma_0")
is_equal(np.cos(2.0*lalsim_coeffs_check_p['zeta']),     xp_tiw.precession_coefficients[None].cos_2zeta, "cos_2zeta")
is_equal(np.sin(2.0*lalsim_coeffs_check_p['zeta']),     xp_tiw.precession_coefficients[None].sin_2zeta, "sin_2zeta")

is_equal(lalsim_coeffs_check_p['alpha1']/np.pi,         xp_tiw.precession_coefficients[None].alpha_m3, "alpha_m3")
is_equal(lalsim_coeffs_check_p['alpha2']/np.pi**(2/3),  xp_tiw.precession_coefficients[None].alpha_m2, "alpha_m2")
is_equal(lalsim_coeffs_check_p['alpha3']/np.pi**(1/3),  xp_tiw.precession_coefficients[None].alpha_m1, "alpha_m1")
is_equal(lalsim_coeffs_check_p['alpha4L'],              xp_tiw.precession_coefficients[None].alpha_log, "alpha_log")
is_equal(lalsim_coeffs_check_p['alpha5']*np.pi**(1/3),  xp_tiw.precession_coefficients[None].alpha_1, "alpha_1")

is_equal(-lalsim_coeffs_check_p['epsilon1']/np.pi,         xp_tiw.precession_coefficients[None].gamma_m3, "gamma_m3")
is_equal(-lalsim_coeffs_check_p['epsilon2']/np.pi**(2/3),  xp_tiw.precession_coefficients[None].gamma_m2, "gamma_m2")
is_equal(-lalsim_coeffs_check_p['epsilon3']/np.pi**(1/3),  xp_tiw.precession_coefficients[None].gamma_m1, "gamma_m1")
is_equal(-lalsim_coeffs_check_p['epsilon4L'],              xp_tiw.precession_coefficients[None].gamma_log, "gamma_log")
is_equal(-lalsim_coeffs_check_p['epsilon5']*np.pi**(1/3),  xp_tiw.precession_coefficients[None].gamma_1, "gamma_1")

is_equal(lalsim_coeffs_check_p['L0']/np.pi**(1/3), xp_tiw.precession_coefficients[None].L_0, "L_0")
is_equal(lalsim_coeffs_check_p['L1'],              xp_tiw.precession_coefficients[None].L_1, "L_1")
is_equal(lalsim_coeffs_check_p['L2']*np.pi**(1/3), xp_tiw.precession_coefficients[None].L_2, "L_2")
is_equal(lalsim_coeffs_check_p['L3']*np.pi**(2/3), xp_tiw.precession_coefficients[None].L_3, "L_3")
is_equal(lalsim_coeffs_check_p['L4']*np.pi,        xp_tiw.precession_coefficients[None].L_4, "L_4")
is_equal(lalsim_coeffs_check_p['L5']*np.pi**(4/3), xp_tiw.precession_coefficients[None].L_5, "L_5")
is_equal(lalsim_coeffs_check_p['L6']*np.pi**(5/3), xp_tiw.precession_coefficients[None].L_6, "L_6")

is_equal(lalsim_coeffs_check_p['Y2m2'], xp_tiw.precession_coefficients[None].spin_minus2_Y2m[0], "Y2m2")
is_equal(lalsim_coeffs_check_p['Y2m1'], xp_tiw.precession_coefficients[None].spin_minus2_Y2m[1], "Y2m1")
is_equal(lalsim_coeffs_check_p['Y20'],  xp_tiw.precession_coefficients[None].spin_minus2_Y2m[2], "Y20")
is_equal(lalsim_coeffs_check_p['Y21'],  xp_tiw.precession_coefficients[None].spin_minus2_Y2m[3], "Y21")
is_equal(lalsim_coeffs_check_p['Y22'],  xp_tiw.precession_coefficients[None].spin_minus2_Y2m[4], "Y22")
# fmt: on

appro_xp = lalsim.GetApproximantFromString("IMRPhenomXP")
extra_params = lal.CreateDict()
lalsim.SimInspiralWaveformParamsInsertPhenomXPrecVersion(extra_params, 102)
hp_lal, hc_lal = lalsim.SimInspiralChooseFDWaveform(
    float(check_params_p["mass_1"] * lal.MSUN_SI),
    float(check_params_p["mass_2"] * lal.MSUN_SI),
    float(check_params_p["chi1_x"]),
    float(check_params_p["chi1_y"]),
    float(check_params_p["chi1_z"]),
    float(check_params_p["chi2_x"]),
    float(check_params_p["chi2_y"]),
    float(check_params_p["chi2_z"]),
    float(check_params_p["luminosity_distance"] * 1e6 * lal.PC_SI),
    float(check_params_p["inclination"]),
    float(check_params_p["reference_phase"]),
    float(long_asc_nodes),
    float(eccentricity),
    float(mean_per_ano),
    float(delta_f),
    float(freqs[0]),
    float(freqs[-1]),
    float(reference_frequency),
    extra_params,
    appro_xp,
)

fig = plot_waveform_comparison(
    freqs,
    xp_tiw.waveform_container_numpy,
    {"plus": hp_lal.data.data[freqs_mask], "cross": hc_lal.data.data[freqs_mask]},
)
fig.tight_layout()

In [ ]:
xp_tiw.update_waveform(check_params_c)

# fmt: off
lalsim_coeffs_check_c = dict(
    chiL = 7.2669572741458710e-01,
    chip = 6.6797538777064158e-01,

    afinal = 9.8653609037362011e-01,
    fring  = 1.3576946169750617e-01,
    fdamp  = 5.2674805641292144e-03,

    alpha_offset   = 5.2050712304625702e+09,
    epsilon_offset = -1.5735877967072774e+03,
    zeta           = -2.6202594554148582e+00,

    alpha1  = -1.0425963737823254e-01,
    alpha2  = -9.5500127492024575e+01,
    alpha3  = -4.9274971049896907e+04,
    alpha4L = -1.0029639081918487e+07,
    alpha5  = 1.4493072900152763e+10,

    epsilon1  = -1.0425963737823254e-01,
    epsilon2  = -9.5500127492024575e+01,
    epsilon3  = -1.1613914504159686e+00,
    epsilon4L = 1.4004072882187057e+02,
    epsilon5  = -1.0921966441556517e+03,

    L0 = 1.0000000000000000e+00,
    L1 = 0.0000000000000000e+00,
    L2 = 1.5001978663060525e+00,
    L3 = -2.4181221474068084e+00,
    L4 = 3.3721804638653659e+00,
    L5 = -5.0713086937874046e+00,
    L6 = 8.4007227834592495e+00,

    Y2m2 = 2.5494732944774463e-02,
    Y2m1 = 1.0164828119497037e-01,
    Y20  = 2.4817912299897701e-01,
    Y21  = 4.0396077119035467e-01,
    Y22  = 4.0265099217395922e-01,
)

is_equal(lalsim_coeffs_check_c['chiL'], xp_tiw.source_parameters[None].chi_l, "chi_l")
is_equal(lalsim_coeffs_check_c['chip'], xp_tiw.source_parameters[None].chi_p, "chi_p")

is_equal(lalsim_coeffs_check_c['afinal'], xp_tiw.source_parameters[None].final_spin, "final_spin")
is_equal(lalsim_coeffs_check_c['fring'],  xp_tiw.source_parameters[None].f_ring, "f_ring")
is_equal(lalsim_coeffs_check_c['fdamp'],  xp_tiw.source_parameters[None].f_damp, "f_damp")

is_equal(-lalsim_coeffs_check_c['alpha_offset'],  xp_tiw.precession_coefficients[None].alpha_0 - xp_tiw.precession_coefficients[None].alpha_log * np.log(np.pi), "alpha_0")
is_equal(lalsim_coeffs_check_c['epsilon_offset'], xp_tiw.precession_coefficients[None].gamma_0 - xp_tiw.precession_coefficients[None].gamma_log * np.log(np.pi), "gamma_0")
is_equal(np.cos(2.0*lalsim_coeffs_check_c['zeta']),     xp_tiw.precession_coefficients[None].cos_2zeta, "cos_2zeta")
is_equal(np.sin(2.0*lalsim_coeffs_check_c['zeta']),     xp_tiw.precession_coefficients[None].sin_2zeta, "sin_2zeta")

is_equal(lalsim_coeffs_check_c['alpha1']/np.pi,         xp_tiw.precession_coefficients[None].alpha_m3, "alpha_m3")
is_equal(lalsim_coeffs_check_c['alpha2']/np.pi**(2/3),  xp_tiw.precession_coefficients[None].alpha_m2, "alpha_m2")
is_equal(lalsim_coeffs_check_c['alpha3']/np.pi**(1/3),  xp_tiw.precession_coefficients[None].alpha_m1, "alpha_m1")
is_equal(lalsim_coeffs_check_c['alpha4L'],              xp_tiw.precession_coefficients[None].alpha_log, "alpha_log")
is_equal(lalsim_coeffs_check_c['alpha5']*np.pi**(1/3),  xp_tiw.precession_coefficients[None].alpha_1, "alpha_1")

is_equal(-lalsim_coeffs_check_c['epsilon1']/np.pi,         xp_tiw.precession_coefficients[None].gamma_m3, "gamma_m3")
is_equal(-lalsim_coeffs_check_c['epsilon2']/np.pi**(2/3),  xp_tiw.precession_coefficients[None].gamma_m2, "gamma_m2")
is_equal(-lalsim_coeffs_check_c['epsilon3']/np.pi**(1/3),  xp_tiw.precession_coefficients[None].gamma_m1, "gamma_m1")
is_equal(-lalsim_coeffs_check_c['epsilon4L'],              xp_tiw.precession_coefficients[None].gamma_log, "gamma_log")
is_equal(-lalsim_coeffs_check_c['epsilon5']*np.pi**(1/3),  xp_tiw.precession_coefficients[None].gamma_1, "gamma_1")

is_equal(lalsim_coeffs_check_c['L0']/np.pi**(1/3), xp_tiw.precession_coefficients[None].L_0, "L_0")
is_equal(lalsim_coeffs_check_c['L1'],              xp_tiw.precession_coefficients[None].L_1, "L_1")
is_equal(lalsim_coeffs_check_c['L2']*np.pi**(1/3), xp_tiw.precession_coefficients[None].L_2, "L_2")
is_equal(lalsim_coeffs_check_c['L3']*np.pi**(2/3), xp_tiw.precession_coefficients[None].L_3, "L_3")
is_equal(lalsim_coeffs_check_c['L4']*np.pi,        xp_tiw.precession_coefficients[None].L_4, "L_4")
is_equal(lalsim_coeffs_check_c['L5']*np.pi**(4/3), xp_tiw.precession_coefficients[None].L_5, "L_5")
is_equal(lalsim_coeffs_check_c['L6']*np.pi**(5/3), xp_tiw.precession_coefficients[None].L_6, "L_6")

is_equal(lalsim_coeffs_check_c['Y2m2'], xp_tiw.precession_coefficients[None].spin_minus2_Y2m[0], "Y2m2")
is_equal(lalsim_coeffs_check_c['Y2m1'], xp_tiw.precession_coefficients[None].spin_minus2_Y2m[1], "Y2m1")
is_equal(lalsim_coeffs_check_c['Y20'],  xp_tiw.precession_coefficients[None].spin_minus2_Y2m[2], "Y20")
is_equal(lalsim_coeffs_check_c['Y21'],  xp_tiw.precession_coefficients[None].spin_minus2_Y2m[3], "Y21")
is_equal(lalsim_coeffs_check_c['Y22'],  xp_tiw.precession_coefficients[None].spin_minus2_Y2m[4], "Y22")
# fmt: on

appro_xp = lalsim.GetApproximantFromString("IMRPhenomXP")
extra_params = lal.CreateDict()
lalsim.SimInspiralWaveformParamsInsertPhenomXPrecVersion(extra_params, 102)
hp_lal, hc_lal = lalsim.SimInspiralChooseFDWaveform(
    float(check_params_c["mass_1"] * lal.MSUN_SI),
    float(check_params_c["mass_2"] * lal.MSUN_SI),
    float(check_params_c["chi1_x"]),
    float(check_params_c["chi1_y"]),
    float(check_params_c["chi1_z"]),
    float(check_params_c["chi2_x"]),
    float(check_params_c["chi2_y"]),
    float(check_params_c["chi2_z"]),
    float(check_params_c["luminosity_distance"] * 1e6 * lal.PC_SI),
    float(check_params_c["inclination"]),
    float(check_params_c["reference_phase"]),
    float(long_asc_nodes),
    float(eccentricity),
    float(mean_per_ano),
    float(delta_f),
    float(freqs[0]),
    float(freqs[-1]),
    float(reference_frequency),
    extra_params,
    appro_xp,
)

fig = plot_waveform_comparison(
    freqs,
    xp_tiw.waveform_container_numpy,
    {"plus": hp_lal.data.data[freqs_mask], "cross": hc_lal.data.data[freqs_mask]},
)
fig.tight_layout()

**Comments for the mismatch results:**

In the range of large mass ratio (m1/m2>100), PhenomXP waveform implemented in tiwave has relatively large mismatch with lalsimulation. This may be mainly caused by the numerical errors of alpha_log and alpha_1 which contain the terms of eta^-3 and eta^-4. If the mass ratio is extreme, these terms can be very large and have large numerical errors. The maximum mismatch in our test samples is in the order of 10^-7, we think it is acceptable in most cases. 